In [19]:
import pathlib
import requests
import subprocess
import os
from typing import List, Tuple

#### The input_file needs to be formatted properly if you do not use the one created from swift_database_creator.ipynb
Space delimited data with each line containing a target name -> observation id (no commas or any other characters between these elements)   
*Note: This data can have any amount of spacing between the name and observation id, just none* ***before the name*** *and/or* ***after the id***

In [20]:
def create_tlist(input_file: str) -> List[Tuple[str, str]]:
    tlist = []
    file = open(f'{input_file}', "r")
    lines = file.readlines()
    i = 0
    while i < len(lines):
        current_line = lines[i]
        obsid = current_line.split(' ')[-1]
        tname = current_line.split(' ')[0]
        if (tname != 'Name' and tname != '-------------------'):
            tlist.append((obsid, tname))
        i += 1
    file.close()
    return tlist

In [21]:
def create_swift_wget_commands(obsid: str, dtype: str, overwrite: bool=False) -> str:
    
    # -nc ==> no clobber: don't replace already downloaded files
    # -q ==> quiet mode, no output
    # -w 2 ==> wait 2 seconds between files
    # -nH ==> don't create a directory based on the host, in this case no folder named www.swift.ac.uk/
    # --cut-dirs=2 ==> remove the /archive/reproc/ folders on the server from being created locally
    # -r ==> recursive: grab everything under this folder on the server
    # --reject ... ==> specify files that we don't want from the server
    
    if overwrite is False:
        overwrite_option = '-nc'
    else:
        overwrite_option = ''
    wget_command = 'wget ' + overwrite_option + f' -q -w 2 -nH --cut-dirs=2 -r --no-parent --reject index.html*,robots.txt*  http://www.swift.ac.uk/archive/reproc/{obsid}/{dtype}'
    return wget_command

In [26]:
def swift_download_uncompressed(obsid: str, tname: str, dtype: str, dest_dir: pathlib.Path = None, overwrite: bool=False) -> None:
    wget_command = create_swift_wget_commands(obsid, dtype, overwrite)
    print(wget_command.split())
    old_cwd = os.getcwd()
    if dest_dir is not None:
        os.chdir(dest_dir)
    print(f"Downloading {dtype} data of observation id {obsid} to {os.getcwd()} ...")
    print(wget_command.split(' ')[-1])
    presult = subprocess.run(wget_command.split())
    if presult.returncode != 0:
        print(f"Non-zero return code {presult.returncode} for {wget_command}!")
    updated_name = tname.replace('/', '_')
    try:
        os.rename(f'{os.getcwd()}/{obsid}', f'{os.getcwd()}/{updated_name}_{obsid}')
        os.chdir(old_cwd)
    except os.error:
        os.chdir(old_cwd)

In [23]:
def swift_download_compressed() -> None:
    return

In [24]:
def download_data(tlist: str, dtype_list: str, dest_dir: pathlib.Path, download_type: str, overwrite=False) -> None:
    for dtype in dtype_list:
        for obsid, tname in tlist:
            if download_type == 'uncompressed':
                swift_download_uncompressed(obsid=obsid, tname=tname, dtype=dtype, dest_dir=dest_dir, overwrite=overwrite)
            if download_type == 'tar':
                swift_download_compressed()
    print(f'\nDownload Completed!')

In [27]:
#dtype_list = ['auxil', 'bat', 'xrt', 'uvot', 'log']
dtype_list = ['auxil']
download_type = ['uncompressed', 'tar']
dest_dir = '/Users/jduffy0121/Desktop/files'
overwrite = False
file_name = 'test.txt'
tlist = create_tlist(input_file=file_name)
#print(os.getcwd())
print(tlist)
download_data(tlist, dtype_list, dest_dir, download_type[0], overwrite)

[('00020405001', 'CometC/2013US10(Catalina)')]
['wget', '-nc', '-q', '-w', '2', '-nH', '--cut-dirs=2', '-r', '--no-parent', '--reject', 'index.html*,robots.txt*', 'http://www.swift.ac.uk/archive/reproc/00020405001/auxil']
http://www.swift.ac.uk/archive/reproc/00020405001/auxil


pathconf: Not a directory


KeyboardInterrupt: 